# Explore here

In [1]:
import pandas as pd

# Configuración
file_id = '1e_DPPpXy0zddc0A1ARU4fx1hO_ZHc1Tn'
url = f'https://drive.google.com/uc?id={file_id}&export=download'

# Carga el DataFrame con parámetros para mejor manejo de datos
try:
    df = pd.read_csv(url)
    
    # Verificación básica
    print(f"¡Archivo cargado correctamente! Dimensiones: {df.shape}")
    print("\nPrimeras 5 filas:")
    print(df.head())
    
    print("\nResumen estadístico:")
    print(df.describe())
    
except Exception as e:
    print(f"Error al cargar el archivo: {e}")
    print("\nSolución alternativa:")
    print("1. Verifica que el archivo esté compartido como 'Cualquier persona con el enlace'")
    print("2. Si el archivo es muy grande, usa: df = pd.read_csv(url, chunksize=10000)")

¡Archivo cargado correctamente! Dimensiones: (48052, 2)

Primeras 5 filas:
                                        ComentarioEs sentimiento
0  Uno de los otros crÃ­ticos ha mencionado que d...    positivo
1  Una pequeÃ±a pequeÃ±a producciÃ³n.La tÃ©cnica ...    positivo
2  PensÃ© que esta era una manera maravillosa de ...    positivo
3  BÃ¡sicamente, hay una familia donde un niÃ±o p...    negativo
4  El """"amor en el tiempo"""" de Petter Mattei ...    positivo

Resumen estadístico:
                                             ComentarioEs sentimiento
count                                               48051       48051
unique                                              48051           2
top     Nadie espera que las pelÃ­culas de Star Trek s...    negativo
freq                                                    1       24041


In [2]:
df

,ComentarioEs,sentimiento
0,Uno de los otros crÃ­ticos ha mencionado que d...,positivo
1,Una pequeÃ±a pequeÃ±a producciÃ³n.La tÃ©cnica ...,positivo
2,PensÃ© que esta era una manera maravillosa de ...,positivo
3,"BÃ¡sicamente, hay una familia donde un niÃ±o p...",negativo
4,"El """"""""amor en el tiempo"""""""" de Petter Mattei ...",positivo
...,...,...
48047,PensÃ© que esta pelÃ­cula hizo un buen trabajo...,positivo
48048,"Mala parcela, mal diÃ¡logo, mala actuaciÃ³n, d...",negativo
48049,Soy catÃ³lica enseÃ±ada en escuelas primarias ...,negativo
48050,Voy a tener que estar en desacuerdo con el com...,negativo
